In [1]:
import random
import json_lines
from langdetect import detect_langs
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import KeyedVectors
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix

LIMIT_INPUT_ROWS= 200
MIN_ENGLISH= 0.0
WNL = WordNetLemmatizer()
REMOVEABLE_CHARS= '[^A-Za-z ]'
ALL_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
ACCEPTABLE_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
MIN_WORD_OCCURRENCES= 1
VEC_SIZE= 5
MAX_GRAMS= 4
AMOUNT_OF_FILTS= 10 # put to 100 probably

number_entries= 0
with open ('reviews.json', 'rb') as f:
    for item in json_lines.reader(f):
        number_entries+= 1

In [2]:
# READ IN INFO
reviews= []; polarity= []; are_early= []
included_indexs= []
j= 0
while j < LIMIT_INPUT_ROWS:
    randint= random.randint(0, number_entries-1)
    if not randint in included_indexs:
        included_indexs.append(randint)
        j+= 1
    
with open('reviews.json', 'rb') as f: # 5,000 entries in reviews.json
    for counter, item in enumerate(json_lines.reader(f)):
        if counter in included_indexs:
            reviews.append(item['text'])
            polarity.append(item['voted_up'])
            are_early.append(item['early_access'])

# END PRODUCTS: reviews, polarity, are_early

In [3]:
# FILTER TEXT BY CHARACTER

import re
new_reviews= []
for i, text in enumerate(reviews):
    text= re.sub(REMOVEABLE_CHARS, '', text)
    if not str.isspace(text) and text != "":
        new_reviews.append(text)
#     else:
#         print("removed ["+text+"]")
# print(reviews)
# print("########################################################################")
# print(new_reviews)
reviews= new_reviews

# END PRODUCTS: reviews

In [4]:
# FILTER TEXT BY LANGUAGE

filtered_reviews= []
filtered_polarity= []
filtered_are_early= []
for i, text in enumerate(reviews):
    try:
        langs = detect_langs(text)
    except:
        print("["+text+"]")
        print("ERROR: langs failed to define itself. Happens occassionally - just rerun")
        pass
    for lang in langs:
        if str(lang)[0:2] == 'en':
            if float(str(lang)[3:]) > MIN_ENGLISH:
                filtered_reviews.append(reviews[i])
                filtered_polarity.append(polarity[i])
                filtered_are_early.append(are_early[i])
reviews= filtered_reviews
polarity= filtered_polarity
are_early= filtered_are_early

#print(reviews)
# END PRODUCTS: reviews, polarity, are_early

In [5]:
# FILTER TEXT BY WORD TYPE

#print(reviews)
filtered_reviews= []
for i, text in enumerate(reviews):
    #print("-------------------------------------")
    new_text= []
    split_text= text.split()
    tags= nltk.pos_tag(split_text)
    #print(tags)
    #print("#############")
    lemd_split_text= []
    for word in split_text:
        lemd_split_text.append(WNL.lemmatize(word))
    tags= nltk.pos_tag(lemd_split_text)
    #print(tags)
    for j, word in enumerate(lemd_split_text):
        if tags[j][1] in ACCEPTABLE_TAGS:
            new_text.append(word)
    filtered_reviews.append(new_text)
reviews= [' '.join([str(elem) for elem in sublist]) for sublist in filtered_reviews]

# END PRODUCTS: reviews, polarity, are_early

In [6]:
# GET REVIEWS AS LIST OF LISTS

no_sentences= 0
reviews_as_ll= [] # ll denoting the list of list data
for text in reviews:
    split_text= text.split()
    reviews_as_ll.append(split_text)
    no_sentences+= 1

# END PRODUCTS: reviews_as_ll, no_sentences

In [7]:
# MAKE REVIEWS LOWER CASER

for i, review in enumerate(reviews_as_ll):
    for j, word in enumerate(review):
        reviews_as_ll[i][j]= word.lower()

# END PRODUCTS: reviews_as_ll

In [8]:
# MODEL RELATED

#X_train, X_test, y_train, y_test= train_test_split(reviews_as_ll, polarity, test_size=0.33, random_state=42)

model= gensim.models.Word2Vec(min_count= MIN_WORD_OCCURRENCES, vector_size=VEC_SIZE)
model.build_vocab(reviews_as_ll)
model.train(reviews_as_ll, total_examples=no_sentences, epochs=model.epochs) # can be a non-repeatable, 1-pass generator

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

# END PRODUCTS: wv

In [9]:
# MAKE FILTERS

filters= []
for i in range(AMOUNT_OF_FILTS):
    filter= []
    n_gram= random.randint(2, MAX_GRAMS)
    for j in range(n_gram):
        filter_layer= []
        for k in range(VEC_SIZE):
            filter_layer.append(random.uniform(-1, 1))
        filter.append(filter_layer)
    filter= np.array([np.array(xi) for xi in filter])
    filters.append(filter)

# END PRODUCTS: filters (list of numpy.ndarray)

In [24]:
# GENERATE FEATURES

# rows, cols = (len(reviews_as_ll), AMOUNT_OF_FILTS)
# greatest_activations = [[0]*cols]*rows # greatest (absolute) filter activation values
greatest_activations= {}
# dict_name[(conv_i, filt_i)] = greatest_conv_float

for review_i, review in enumerate(reviews_as_ll):
    for filt_i, filt in enumerate(filters):
        filt= filt.ravel()
        greatest_conv_value= 0
        n_gram= round(filt.size / VEC_SIZE) # round being used to convert to int
        convolutions_per_filt= len(review) - n_gram + 1
        for convolution_i in range(convolutions_per_filt):
            vec_matrix= []
            for i in range(n_gram):
                vec_matrix.append(wv[review[convolution_i + i]]) # WV is a dict
            vec_matrix= np.array([np.array(xi) for xi in vec_matrix]).ravel() # list of lists to 1d nd numpy
            conv_result= np.convolve(filt, vec_matrix, 'valid')
            if abs(conv_result) > abs(greatest_conv_value):
                greatest_conv_value= conv_result
        if type(greatest_conv_value) != int:
            greatest_conv_value= greatest_conv_value[0].astype(float).item()
        if type(greatest_conv_value) == int:
            greatest_conv_value= np.float64(greatest_conv_value).item()
        #print(type(greatest_conv_value))
        #greatest_activations[review_i][filt_i]= greatest_conv_value
        greatest_activations[(review_i, filt_i)]= greatest_conv_value
        print(review_i, filt_i, " = ", greatest_conv_value)
        #print(greatest_conv_value)
# END PRODUCTS: greatest_activations

0 0  =  1.9982476885926408
0 1  =  -0.7943488051027554
0 2  =  0.758992387833686
0 3  =  2.357522640856744
0 4  =  -1.2678337349975544
0 5  =  -3.62898737934907
0 6  =  2.465375884213244
0 7  =  -1.253485176108696
0 8  =  0.8071710583398076
0 9  =  -2.4892200918318155
1 0  =  0.7396810968227777
1 1  =  -0.4650921039032365
1 2  =  0.5610744444273617
1 3  =  1.9026345330860706
1 4  =  -0.964786213680186
1 5  =  -1.8263377628471813
1 6  =  2.059503820851271
1 7  =  -0.6522433069189039
1 8  =  0.4459577304172555
1 9  =  -2.163570506478303
2 0  =  1.631556128244598
2 1  =  -0.7003074262631127
2 2  =  0.6581916360570008
2 3  =  1.8196344261496207
2 4  =  -1.0523107687183708
2 5  =  -3.406094841918576
2 6  =  2.357382046900999
2 7  =  -1.0725154173721925
2 8  =  -0.6793093381078366
2 9  =  -2.04862130333919
3 0  =  1.844338941525181
3 1  =  -0.2734080904323444
3 2  =  0.6218622867773935
3 3  =  0.2757028761764898
3 4  =  0.5269414188940147
3 5  =  -2.1870070519850677
3 6  =  1.791771028068576

34 1  =  -0.8405926333262713
34 2  =  0.7656846538981575
34 3  =  -2.440246202607218
34 4  =  -1.4137774748031133
34 5  =  -3.5454466903373523
34 6  =  2.2898290829437378
34 7  =  -1.1465393851630128
34 8  =  0.8051873803293732
34 9  =  -2.940215983239871
35 0  =  1.9985795492347525
35 1  =  -0.6158819823104315
35 2  =  0.6664923053570943
35 3  =  2.2089277746065363
35 4  =  -1.2770752671468768
35 5  =  -3.5224634406915616
35 6  =  2.4446827505924866
35 7  =  -1.1135650536164825
35 8  =  0.8638733559452833
35 9  =  -2.5410481465491803
36 0  =  2.041548410026594
36 1  =  -0.22234640735526426
36 2  =  0.5969368309739916
36 3  =  0.5080399651217008
36 4  =  -0.8993270806110266
36 5  =  -1.5628326038873608
36 6  =  1.4071554409300475
36 7  =  -0.5801050136715127
36 8  =  -0.29637099474985
36 9  =  -1.5557371280687342
37 0  =  0.04165292365590963
37 1  =  0.0989522220444303
37 2  =  -0.10532438756501768
37 3  =  0.0
37 4  =  0.0
37 5  =  0.0
37 6  =  0.01926194638291511
37 7  =  -0.02780848

63 7  =  -0.8616432382523825
63 8  =  -0.6338822739984653
63 9  =  -2.4791475932999956
64 0  =  1.420821733708568
64 1  =  -0.6240752349685931
64 2  =  0.6574402383922945
64 3  =  1.638494623436836
64 4  =  -0.9966840252105774
64 5  =  -2.802631401637286
64 6  =  2.2822774636482746
64 7  =  -1.13151460026606
64 8  =  0.675362099140588
64 9  =  -2.3794014140734996
65 0  =  0.8121593633081805
65 1  =  -0.45316926704468957
65 2  =  0.34562200328840015
65 3  =  1.2230135602102299
65 4  =  -0.5888523417258636
65 5  =  -1.6752605997173593
65 6  =  1.424939610837977
65 7  =  -0.595560744837103
65 8  =  -0.4199521625891809
65 9  =  -0.9312532778950497
66 0  =  2.0672221168955374
66 1  =  -0.784035568903004
66 2  =  0.8129598847664512
66 3  =  -2.6564014835152605
66 4  =  -1.6303380516121722
66 5  =  -3.534999276619597
66 6  =  2.6106532079140923
66 7  =  -1.3113781708470325
66 8  =  0.8740710588200578
66 9  =  -3.045621143571025
67 0  =  1.9165833861181993
67 1  =  -0.666502731747634
67 2  =  

In [25]:
# for i in range(len(greatest_activations)):
#     print("\nFitler activations",greatest_activations[i])
#     print(polarity[i])
print(greatest_activations)

{(0, 0): 1.9982476885926408, (0, 1): -0.7943488051027554, (0, 2): 0.758992387833686, (0, 3): 2.357522640856744, (0, 4): -1.2678337349975544, (0, 5): -3.62898737934907, (0, 6): 2.465375884213244, (0, 7): -1.253485176108696, (0, 8): 0.8071710583398076, (0, 9): -2.4892200918318155, (1, 0): 0.7396810968227777, (1, 1): -0.4650921039032365, (1, 2): 0.5610744444273617, (1, 3): 1.9026345330860706, (1, 4): -0.964786213680186, (1, 5): -1.8263377628471813, (1, 6): 2.059503820851271, (1, 7): -0.6522433069189039, (1, 8): 0.4459577304172555, (1, 9): -2.163570506478303, (2, 0): 1.631556128244598, (2, 1): -0.7003074262631127, (2, 2): 0.6581916360570008, (2, 3): 1.8196344261496207, (2, 4): -1.0523107687183708, (2, 5): -3.406094841918576, (2, 6): 2.357382046900999, (2, 7): -1.0725154173721925, (2, 8): -0.6793093381078366, (2, 9): -2.04862130333919, (3, 0): 1.844338941525181, (3, 1): -0.2734080904323444, (3, 2): 0.6218622867773935, (3, 3): 0.2757028761764898, (3, 4): 0.5269414188940147, (3, 5): -2.187007

In [26]:
clf= MLPClassifier(random_state=1, max_iter=300).fit(greatest_activations, polarity)
y_pred= clf.predict(greatest_activations)

ValueError: Expected 2D array, got scalar array instead:
array={(0, 0): 1.9982476885926408, (0, 1): -0.7943488051027554, (0, 2): 0.758992387833686, (0, 3): 2.357522640856744, (0, 4): -1.2678337349975544, (0, 5): -3.62898737934907, (0, 6): 2.465375884213244, (0, 7): -1.253485176108696, (0, 8): 0.8071710583398076, (0, 9): -2.4892200918318155, (1, 0): 0.7396810968227777, (1, 1): -0.4650921039032365, (1, 2): 0.5610744444273617, (1, 3): 1.9026345330860706, (1, 4): -0.964786213680186, (1, 5): -1.8263377628471813, (1, 6): 2.059503820851271, (1, 7): -0.6522433069189039, (1, 8): 0.4459577304172555, (1, 9): -2.163570506478303, (2, 0): 1.631556128244598, (2, 1): -0.7003074262631127, (2, 2): 0.6581916360570008, (2, 3): 1.8196344261496207, (2, 4): -1.0523107687183708, (2, 5): -3.406094841918576, (2, 6): 2.357382046900999, (2, 7): -1.0725154173721925, (2, 8): -0.6793093381078366, (2, 9): -2.04862130333919, (3, 0): 1.844338941525181, (3, 1): -0.2734080904323444, (3, 2): 0.6218622867773935, (3, 3): 0.

In [27]:
print("overall accuracy: ", clf.score(greatest_activations, polarity))
fig= plot_confusion_matrix(clf, greatest_activations, polarity)
fig.figure_.suptitle("Confusion Matrix")

ValueError: Expected 2D array, got scalar array instead:
array={(0, 0): 1.9982476885926408, (0, 1): -0.7943488051027554, (0, 2): 0.758992387833686, (0, 3): 2.357522640856744, (0, 4): -1.2678337349975544, (0, 5): -3.62898737934907, (0, 6): 2.465375884213244, (0, 7): -1.253485176108696, (0, 8): 0.8071710583398076, (0, 9): -2.4892200918318155, (1, 0): 0.7396810968227777, (1, 1): -0.4650921039032365, (1, 2): 0.5610744444273617, (1, 3): 1.9026345330860706, (1, 4): -0.964786213680186, (1, 5): -1.8263377628471813, (1, 6): 2.059503820851271, (1, 7): -0.6522433069189039, (1, 8): 0.4459577304172555, (1, 9): -2.163570506478303, (2, 0): 1.631556128244598, (2, 1): -0.7003074262631127, (2, 2): 0.6581916360570008, (2, 3): 1.8196344261496207, (2, 4): -1.0523107687183708, (2, 5): -3.406094841918576, (2, 6): 2.357382046900999, (2, 7): -1.0725154173721925, (2, 8): -0.6793093381078366, (2, 9): -2.04862130333919, (3, 0): 1.844338941525181, (3, 1): -0.2734080904323444, (3, 2): 0.6218622867773935, (3, 3): 0.